# Import Libraries

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.callbacks import TensorBoard

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [2]:
mp_holistic = mp.solutions.holistic #hollistic model
mp_drawing = mp.solutions.drawing_utils #drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert BGR image to RGB for processing
    image.flags.writeable = False
    results = model.process(image) # predict
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # convert image back to BGR and return
    return image, results

In [4]:
# Gives the connections that is detected in the feed (eg: left eye connected to nose)
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
#customized landmarks
def draw_styled_landmarks(image, results):
    #face connection
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness = 1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121), thickness = 1, circle_radius=1))
    #pose connection
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10), thickness = 2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44,121), thickness = 2, circle_radius=2))
    #left hand connection
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76), thickness = 2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121,44,250), thickness = 2, circle_radius=2))
    #right hand connection
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66), thickness = 2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness = 2, circle_radius=2))

In [6]:
# draw_landmarks(frame, results)

In [7]:
# plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [8]:
# get the points
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(33)
    face = np.array([[res.x, res.y, res.z, res.visibility] for res in results.face_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(468)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, left_hand, right_hand])

In [9]:
# result_test = extract_keypoints(results)
# np.save('0', result_test)

In [10]:
DATA_PATH = os.path.join('MP_Data')

actions = np.array(['Hello', 'Thanks', 'I Love You'])

no_sequences = 30

sequence_length = 30

# Create directories for the data to be collected

In [11]:
# for action in actions:
#     for sequence in range(no_sequences):
#         try:
#             os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
#         except:
#             pass

# Record data for keypoints in hands and body pose and place them in the folders created in previous block

In [12]:
# cap = cv2.VideoCapture(0)
# with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence=0.5) as holistic:
    
#     for action in actions:
#         for sequence in range(no_sequences):
#             for frame_num in range(sequence_length):
        
#                 ret, frame = cap.read()

#                 #made detection
#                 image, results = mediapipe_detection(frame, holistic)
#                 #draw landmarks
#                 draw_styled_landmarks(image, results)

#                 if frame_num == 0:
#                     cv2.putText(image, 'STARTING COLLECTION', (120,200),
#                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
#                     cv2.putText(image, 'Collecting frames for {} Video number {}'.format(action,sequence), (15,12),
#                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, cv2.LINE_AA)
#                     cv2.imshow('OpenCV Feed', image)
#                     cv2.waitKey(2000)
#                 else:
#                     cv2.putText(image, 'Collecting frames for {} Video number {}'.format(action, sequence), (15,12),
#                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, cv2.LINE_AA) 
#                     cv2.imshow('OpenCV Feed', image)

#                 #keypoints
#                 keypoints = extract_keypoints(results)
#                 npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
#                 np.save(npy_path, keypoints)
                
#                 if cv2.waitKey(10) & 0xFF == ord('q'):
#                     break
#     cap.release()
#     cv2.destroyAllWindows()

In [13]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'Hello': 0, 'Thanks': 1, 'I Love You': 2}

# Load data form the directories into 'sequences' array

In [14]:
sequences, labels = [], []

for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [15]:
np.array(sequences).shape

(90, 30, 2130)

In [16]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [18]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(85, 30, 2130) (5, 30, 2130) (85, 3) (5, 3)


In [19]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

# Simple LSTM

In [20]:
model = Sequential()
model.add(LSTM(64, return_sequences = False, activation='relu', input_shape=(30,2130)))
model.add(Dense(actions.shape[0], activation='softmax'))

# Bidirectional LSTM

In [ ]:
# model = Sequential()
# model.add(Bidirectional(LSTM(64, return_sequences = False, activation='relu', input_shape=(30,2130))))
# model.add(Dense(actions.shape[0], activation='softmax'))

# Stacked LSTM network:

In [ ]:
# model = Sequential()
# model.add(LSTM(64, return_sequences = True, activation='relu', input_shape=(30,2130)))
# model.add(LSTM(128, return_sequences = True, activation='relu'))
# model.add(LSTM(64, return_sequences = False, activation='relu'))

# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(actions.shape[0], activation='softmax'))

In [21]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [22]:
model.fit(x_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
3/3 [==============================] - 2s 174ms/step - loss: 1.6161 - categorical_accuracy: 0.1292
Epoch 2/100
3/3 [==============================] - 0s 62ms/step - loss: 1.2656 - categorical_accuracy: 0.5832
Epoch 3/100
3/3 [==============================] - 0s 64ms/step - loss: 1.2057 - categorical_accuracy: 0.4324
Epoch 4/100
3/3 [==============================] - 0s 61ms/step - loss: 0.8398 - categorical_accuracy: 0.6812
Epoch 5/100
3/3 [==============================] - 0s 65ms/step - loss: 1.1993 - categorical_accuracy: 0.7161
Epoch 6/100
3/3 [==============================] - 0s 64ms/step - loss: 0.7448 - categorical_accuracy: 0.7201
Epoch 7/100
3/3 [==============================] - 0s 37ms/step - loss: 0.9328 - categorical_accuracy: 0.5870
Epoch 8/100
3/3 [==============================] - 0s 35ms/step - loss: 0.6795 - categorical_accuracy: 0.6673
Epoch 9/100
3/3 [==============================] - 0s 37ms/step - loss: 0.6661 - categorical_accuracy: 0.5871
Epoch 10/

3/3 [==============================] - 0s 40ms/step - loss: 0.0014 - categorical_accuracy: 1.0000
Epoch 75/100
3/3 [==============================] - 0s 37ms/step - loss: 0.0012 - categorical_accuracy: 1.0000
Epoch 76/100
3/3 [==============================] - 0s 40ms/step - loss: 0.0012 - categorical_accuracy: 1.0000
Epoch 77/100
3/3 [==============================] - 0s 37ms/step - loss: 6.6720e-04 - categorical_accuracy: 1.0000
Epoch 78/100
3/3 [==============================] - 0s 33ms/step - loss: 6.4449e-04 - categorical_accuracy: 1.0000
Epoch 79/100
3/3 [==============================] - 0s 34ms/step - loss: 4.4599e-04 - categorical_accuracy: 1.0000
Epoch 80/100
3/3 [==============================] - 0s 36ms/step - loss: 8.0244 - categorical_accuracy: 0.8902
Epoch 81/100
3/3 [==============================] - 0s 40ms/step - loss: 21.2588 - categorical_accuracy: 0.3993
Epoch 82/100
3/3 [==============================] - 0s 36ms/step - loss: 25.5536 - categorical_accuracy: 0.3288


In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                561920    
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 562,115
Trainable params: 562,115
Non-trainable params: 0
_________________________________________________________________


In [24]:
result = model.predict(x_test)

In [25]:
model.save('action.h5')

In [26]:
model.load_weights('action.h5')

# Evaluate

In [27]:
yhat = model.predict(x_test)

In [28]:
y_true = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [29]:
y_true

[1, 1, 0, 0, 1]

In [30]:
multilabel_confusion_matrix(y_true, yhat)

array([[[3, 0],
        [0, 2]],

       [[2, 0],
        [0, 3]]], dtype=int64)

In [31]:
accuracy_score(y_true, yhat)

1.0

# Real time detection

In [32]:
colors = [(245,117,16), (117,245,16), (18,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        if num >= 3:
            break
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*200), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

# The below block can be run if the data is recorded (if block 12 is run) then we have variable 'image' initialized and we could run the following block to visualize the training phase

In [33]:
# plt.figure(figsize=(18,18))
# plt.imshow(prob_viz(res, actions, image, colors))

In [37]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))

        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 

                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(3) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [35]:
cap.release()
cv2.destroyAllWindows()